In [91]:
import pandas as pd
import numpy as np
import math
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split


In [105]:
file_path = 'creditcard.csv'  # Update the file path if needed
data = pd.read_csv(file_path)

In [107]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [129]:
# normalize the data
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
df = data
# The function "len" counts the number of classes = 1 and saves it as an object "fraud_records"
fraud_records = len(df[df.Class == 1])

# Defines the index for fraud and non-fraud in the lines:
fraud_indices = df[df.Class == 1].index
not_fraud_indices = df[df.Class == 0].index

# Randomly collect equal samples of each type:
under_sample_indices = np.random.choice(not_fraud_indices, fraud_records, False)
df_undersampled = df.iloc[np.concatenate([fraud_indices, under_sample_indices]),:]
X_undersampled = df_undersampled.iloc[:,1:30]
Y_undersampled = df_undersampled.Class
X_undersampled_train, X_undersampled_test, Y_undersampled_train, Y_undersampled_test = train_test_split(X_undersampled, Y_undersampled, test_size = 0.3)




In [130]:
# check the size of X_undersampled_train and X_undersampled_test
print(X_undersampled_train.shape)
print(X_undersampled_test.shape)
print(Y_undersampled_train.shape)
print(Y_undersampled_test.shape)

(688, 29)
(296, 29)
(688,)
(296,)


In [131]:
# split the data into train and test 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=123)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [116]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = n_iterations
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        # Clip the input values to avoid overflow in the exponential function
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))


    def fit(self, X, y):
        num_samples, num_features = X.shape
        self.weights = np.zeros(num_features)
        self.bias = 0

        for _ in range(self.num_iterations):
            model = np.dot(X, self.weights) + self.bias
            predictions = self._sigmoid(model)

            dw = (1 / num_samples) * np.dot(X.T, (predictions - y))
            db = (1 / num_samples) * np.sum(predictions - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        model = np.dot(X, self.weights) + self.bias
        predictions = self._sigmoid(model)
        return [1 if i > 0.5 else 0 for i in predictions]


In [124]:
def compute_gradient(X, y, w, b):
   gradient_w = np.dot(X.T, (np.dot(X, w) + b - y)) / y.size
   gradient_b = np.sum(np.dot(X, w) + b - y) / y.size
   return gradient_w, gradient_b

def update_parameters(w, b, gradient_w, gradient_b, learning_rate):
   w = w - learning_rate * gradient_w
   b = b - learning_rate * gradient_b
   return w, b

def logistic_regression(X, y, w, b, learning_rate, num_iterations):
   for i in range(num_iterations):
       gradient_w, gradient_b = compute_gradient(X, y, w, b)
       w, b = update_parameters(w, b, gradient_w, gradient_b, learning_rate)
   return w, b

def predict(X, w, b):
   return 1 / (1 + np.exp(-(np.dot(X, w) + b)))

w = np.zeros(X_undersampled_train.shape[1])
b = 0

learning_rate = 0.01
num_iterations = 1000

w, b = logistic_regression(X_undersampled_train, Y_undersampled_train, w, b, learning_rate, num_iterations)
# predict
y_pred = predict(X_undersampled_test, w, b)
y_pred = np.where(y_pred > 0.6, 1, 0)

# evaluate
print('Accuracy:', accuracy_score(Y_undersampled_test, y_pred))
print('Precision:', precision_score(Y_undersampled_test, y_pred))
print('Recall:', recall_score(Y_undersampled_test, y_pred))
print('F1:', f1_score(Y_undersampled_test, y_pred))
print('AUC:', roc_auc_score(Y_undersampled_test, y_pred))
print('Confusion Matrix:', confusion_matrix(Y_undersampled_test, y_pred))

Accuracy: 0.5067567567567568
Precision: 0.0
Recall: 0.0
F1: 0.0
AUC: 0.5
Confusion Matrix: [[150   0]
 [146   0]]


/Users/Fazil/opt/anaconda3/envs/mlp/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [125]:
# train the model
model = LogisticRegression(learning_rate=0.01, n_iterations=1000)
model.fit(X_undersampled_train, Y_undersampled_train)

In [127]:
# make predictions
y_pred = model.predict(X_undersampled_test)

In [128]:
# get the metrics
print("Accuracy:", accuracy_score(Y_undersampled_test, y_pred))
print("Precision:", precision_score(Y_undersampled_test, y_pred))
print("Recall:", recall_score(Y_undersampled_test, y_pred))
print("F1 Score:", f1_score(Y_undersampled_test, y_pred))
print("ROC AUC Score:", roc_auc_score(Y_undersampled_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(Y_undersampled_test, y_pred))

Accuracy: 0.8783783783783784
Precision: 0.9230769230769231
Recall: 0.821917808219178
F1 Score: 0.8695652173913043
ROC AUC Score: 0.8776255707762557
Confusion Matrix:
 [[140  10]
 [ 26 120]]


In [44]:
# Use sklearn's LogisticRegression model
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

In [45]:
# get the metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred)) 

Accuracy: 0.9867886178861789
Precision: 1.0
Recall: 0.8452380952380952
F1 Score: 0.9161290322580644
ROC AUC Score: 0.9226190476190477
